# exercises

1.  1\. list all customers (full names, customer ID, and country) who are not in the US
2.   2. list all customers from brazil
3.   3. list all sales agents
4.  4\. show a list of all countries in billing addresses on invoices.
5.   5. how many invoices were there in 2009, and what was the sales total for that year?
6.  \--    (extra challenge: find the invoice count sales total for every year, using one query)
7.  6\. how many line items were there for invoice #37?
8.   7. how many invoices per country?
9.   8. show total sales per country, ordered by highest sales first.

In [ ]:
SELECT [CustomerId],[FirstName],[LastName] ,[Country]
FROM [dbo].[Customer]
WHERE Country != 'United States'

In [ ]:
SELECT *
FROM Customer
WHERE Country='Brazil'

In [ ]:
SELECT *
FROM Employee
WHERE Title='Sales Support Agent'

In [ ]:
SELECT DISTINCT BillingCountry as Country
FROM Invoice

In [ ]:
SELECT YEAR(InvoiceDate) as InvoiceYear, COUNT(*) as SalesCount2009, SUM(Total) as Total
FROM Invoice
GROUP BY YEAR(InvoiceDate)
--HAVING YEAR(InvoiceDate) = 2009

In [ ]:
SELECT COUNT(*) as cnt
FROM InvoiceLine
GROUP BY InvoiceId
HAVING InvoiceId = 37

In [ ]:
SELECT BillingCountry, count(*) as N_Invoices
FROM Invoice
GROUP BY BillingCountry

In [ ]:
SELECT BillingCountry, SUM(Total) as Total
FROM Invoice
GROUP BY BillingCountry
ORDER BY Total DESC

# join exercises (Chinook database)

1.  \-- 1. show all invoices of customers from Brazil (mailing address not billing)
2.  \-- 2. show all invoices together with the name of the sales agent of each one
3.  \-- 3. show all playlists ordered by the total number of tracks they have
4.  \-- 4. which sales agent made the most in sales in 2009?
5.  \-- 5. how many customers are assigned to each sales agent?
6.  \-- 6. which track was purchased the most since 2010?
7.  \-- 7. show the top three best-selling artists.
8.  \-- 8. which customers have the same initials as at least one other customer?

In [ ]:
SELECT * 
FROM Invoice
INNER JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
WHERE Customer.Country = 'Brazil'

In [ ]:
SELECT TOP (10) Invoice.InvoiceId, Employee.FirstName, Employee.LastName, Employee.EmployeeId
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
JOIN Employee ON Customer.SupportRepId = Employee.EmployeeId

In [ ]:
SELECT Playlist.PlaylistId, Name, ISNULL(N_tracks, 0 ) AS cnt
FROM Playlist
LEFT JOIN (
    SELECT PlaylistId, count(*) as N_tracks
    FROM PlaylistTrack
    GROUP BY PlaylistId
    ) as TrackCounts
    ON Playlist.PlaylistId = TrackCounts.PlaylistId
ORDER BY N_tracks DESC 

In [ ]:
SELECT TOP (1) Employee.*, N_Sales
FROM Employee 
INNER JOIN Customer On EmployeeId = SupportRepId
INNER JOIN  (
    SELECT CustomerId, Count(*) as N_Sales
    FROM Invoice
    GROUP BY CustomerId) as SalesCount
ON Customer.CustomerId = SalesCount.CustomerId
ORDER BY N_Sales DESC

In [ ]:
SELECT N_Customers, Employee.*
FROM Employee
INNER JOIN (
    SELECT SupportRepId as EmployeeId, COUNT(*) as N_Customers
    FROM Customer
    GROUP BY SupportRepId
) as CustomerCount
ON CustomerCount.EmployeeId = Employee.EmployeeId

In [ ]:
WITH TrackCount (N_Purchases, TrackId)
AS (
    SELECT COUNT(*) as N_Purchases, TrackId
    FROM InvoiceLine
    INNER JOIN (
        SELECT InvoiceId
        FROM Invoice
        WHERE YEAR(InvoiceDate) >= 2010
    ) AS InviceRecent
    ON InviceRecent.InvoiceId = InvoiceLine.InvoiceId
    GROUP BY TrackId
)
SELECT N_Purchases, Track.*
FROM Track 
INNER JOIN TrackCount
ON TrackCount.TrackId = Track.TrackId
INNER JOIN (
    SELECT TOP (1) N_Purchases as Max_Count
    FROM TrackCount
    ORDER BY N_Purchases DESC
) AS MC
ON MC.Max_Count = TrackCount.N_Purchases

In [ ]:
SELECT TOP (3) Composer, COUNT(*) as N_Sales
FROM Track
GROUP BY Composer
HAVING Composer IS NOT NULL
ORDER BY N_Sales DESC

In [ ]:
SELECT *
FROM Customer as c1
CROSS JOIN Customer as c2
WHERE c1.CustomerId != c2.CustomerId 
AND LEFT(c1.FirstName, 1) = LEFT(c2.FirstName, 1)
AND LEFT(c1.LastName, 1) = LEFT(c2.LastName, 1)

# \-- subquery exercises

solve these with a mixture of joins, subqueries, CTE, and set operators.  

solve at least one of them in two different ways, and see if the execution 

plan for them is the same, or different.

1.  \-- 1. which artists did not make any albums at all?
2.  \-- 2. which artists did not record any tracks of the Latin genre?
3.  \-- 3. which video track has the longest length? (use media type table)
4.  \-- 4. find the names of the customers who live in the same city as the boss employee (the one who reports to nobody)
5.  \-- 5. how many audio tracks were bought by German customers, and what was  the total price paid for them?
6.  \-- 6. list the names and countries of the customers supported by an employee who was hired younger than 35.

In [ ]:
SELECT Artist.ArtistId, Artist.Name
FROM Artist
FULL OUTER JOIN Album
ON Artist.ArtistId = Album.ArtistId
GROUP BY Artist.ArtistId, Artist.Name
HAVING COUNT(AlbumId) = 0

In [ ]:
SELECT Artist.ArtistId, Artist.Name
FROM Track
INNER JOIN (
    SELECT GenreId
    FROM Genre
    WHERE Genre.Name = 'Latin'
) AS LATIN 
ON LATIN.GenreId = Track.GenreId
INNER JOIN Album ON Album.AlbumId = Track.AlbumId
RIGHT JOIN Artist ON Artist.ArtistId = Album.ArtistId
GROUP BY Artist.ArtistId, Artist.Name
HAVING COUNT(TrackId) = 0

In [ ]:
SELECT Track.*
FROM Track
INNER JOIN (
    SELECT MAX(Milliseconds) as Mx_RunTime
    FROM Track
    GROUP BY MediaTypeId
    HAVING MediaTypeId = 3
) AS M_Length
ON Track.Milliseconds = M_Length.Mx_RunTime

In [ ]:
SELECT *
FROM Employee 
LEFT JOIN (
    SELECT City
    FROM Employee
    WHERE ReportsTo is NULL
) AS BossCity
ON BossCity.City = Employee.City

In [ ]:
SELECT SUM(Invoice.Total) as Cost, Count(InvoiceLineId) as PurchaseCount
FROM InvoiceLine
INNER JOIN Invoice ON InvoiceLine.InvoiceId = Invoice.InvoiceId
INNER JOIN Customer ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.Country
HAVING Customer.Country = 'Germany'

In [ ]:
SELECT Customer.*
FROM Customer 
INNER JOIN (
    SELECT EmployeeId
    FROM Employee
    WHERE DATEDIFF(YEAR, Employee.BirthDate, Employee.HireDate) < 35
    AND Title = 'Sales Support Agent'
) AS YG_AG
ON Customer.SupportRepId = YG_AG.EmployeeId